In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import run_full_analysis, print_continuous_status, download_analysis

### Define task function

#### Standard config

In [ ]:
ISO3 = "BIH"
ANALYSIS = "main"

N_CHAINS = 32
N_OPTI_SEARCHES = 8
OPTI_BUDGET = 10000

METROPOLIS_TUNE = 2000
METROPOLIS_DRAWS = 10000
METROPOLIS_METHOD = "DEMetropolis"

FULL_RUNS_SAMPLES = 1000
BURN_IN = 5000

#### Testing config

In [ ]:
# ISO3 = "FRA"
# ANALYSIS = "main"

# N_CHAINS = 32
# N_OPTI_SEARCHES = 8
# OPTI_BUDGET = 700

# METROPOLIS_TUNE = 200
# METROPOLIS_DRAWS = 1000
# METROPOLIS_METHOD = "DEMetropolis"

# FULL_RUNS_SAMPLES = 100
# BURN_IN = 500

### Check that configuration is relevant

In [ ]:
assert N_CHAINS % N_OPTI_SEARCHES == 0
assert (METROPOLIS_DRAWS - BURN_IN) * N_CHAINS >= FULL_RUNS_SAMPLES
assert METROPOLIS_METHOD in ("DEMetropolis", "DEMetropolisZ")

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        opti_params={'n_searches': N_OPTI_SEARCHES, 'num_workers': 8, 'parallel_opti_jobs': 8, 'warmup_iterations': 0, 'search_iterations': OPTI_BUDGET, 'init_method': 'LHS'},
        mcmc_params={'draws': METROPOLIS_DRAWS, 'tune': METROPOLIS_TUNE, 'cores': N_CHAINS, 'chains': N_CHAINS, 'method': METROPOLIS_METHOD},
        full_run_params={'samples': FULL_RUNS_SAMPLES, 'burn_in': BURN_IN},
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(N_CHAINS, 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
analysis_title = "full_run"
config_str = f"_{ANALYSIS}_LHS{N_OPTI_SEARCHES}_opt{OPTI_BUDGET}_mc{METROPOLIS_TUNE}n{METROPOLIS_DRAWS}"

run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, analysis_title + config_str)
run_path

In [ ]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
runner.s3.get_status()

In [ ]:
print(runner.top())

In [ ]:
# wait function with status printing
print_continuous_status(runner)

In [ ]:
# run_path = 'projects/school_project/BIH/2023-07-20T1522-full_run_main_LHS8_opt10000_mc2000n10000'

In [ ]:
download_analysis(run_path)